##  知然算法【2】灰色模型GM(1,N)



### 0、符号说明

+ 原始数据序列：$x_{i}^{0}(k), k=1,2,……,n;n$为数据样本条数,也就是时间序列的长度; $i=1,2,……,N;$其中$N$为数据特征数;$i=1$的序列为目标数据序列；
+ 一次累加数据序列：$x_{i}^{1}(k)=\sum_{j=1}^{k}x_{i}^{0}(j), k=1,2,……,n；i=1,2,……,N$

+ 累加数据预测序列：$\hat{x}_{1}^{1}(k), k=1,2,……,n$


### 1、模型背景

白色系统：系统的内部信息是完全已知的。黑色系统：系统的内部信息是完全未知的，只能通过它与外界的联系来加以观测研究，也就是可以知道该系统的输入输出关系，但不知道它内部是如何实现这种关系的。灰色系统：介于前面两者之间，系统内的一部分信息是已知的，另一部分信息是未知的。

灰色系统理论是由我国著名学者邓聚龙教授于1982年创立的。该理论是将具有潜在规律的时间序列数据经累加生成后，使其变为具有指数增长规律的上升形状数列，而一阶微分方程解的形式是指数增长形式，所以可对累加生成后的数列建立微分方程模型。



### 2、模型步骤

要求每一条样本的时间间隔必须是一致的，并且不存在缺失值。

+ **STEP1:** 当N大于2时，考虑到非目标序列的不同特征的量纲可能不同，需要对特征数据序列进行均值化(列数据除以列均值)或者初值化(列数据除以列第一个值)；
+ **STEP2:** 对目标数据序列进行级比检验，当所有级比值$\lambda(j)$在区间$[e^{-2/(n+1)}, e^{2/(n+1)}]$内，可建立模型；否则需要对数据进行平移转
换，也就是将该数据序列同时加上一个适当的系数$\beta$(需要保存该系数，预测时应再减去该系数)，使得级比值全部在区间内，其中级比值：
$$\lambda(j) = \frac{x_{1}^{0}(j-1)}{x_{1}^{0}(j)}, j=2,3,……,n$$
此时新构成的目标数据序列为：
$$x_{i}^{0}(k)=x_{i}^{0}(k)+\beta$$

+ **STEP3:** 针对所有的特征序列计算一次累加数据序列$x_{i}^{1}(k)$；
+ **STEP4:** 对目标数据序列的一次累加序列计算均值序列：
$$Z^{1}(k) = \frac{x_{1}^{0}(k-1)+x_{1}^{0}(k)}{2}, k=2,3,……,n$$

+ **STEP5:** 计算模型参数：

$$Y=\begin{bmatrix} x_{1}^{0}(2) \\ x_{1}^{0}(3) \\ ……  \\x_{1}^{0}(n)\end{bmatrix}$$

(1)**当$N= 1$时，也就是特征数据只有目标数据序列**
  
$$B = \begin{bmatrix} -Z^{1}(2) & 1 \\ -Z^{1}(3) & 1 \\ …… & 1 \\-Z^{1}(n) & 1 \end{bmatrix}$$
       
       
(2)**当$N\geq 2$时:**

$$B = \begin{bmatrix} -Z^{1}(2) & x_{2}^{1}(2)  & …… & x_{N}^{1}(2)\\ -Z^{1}(3) & x_{2}^{1}(3)  & …… & x_{N}^{1}(3)\\ …… &  ……& …… &  ……\\-Z^{1}(n) & x_{2}^{1}(n)  & …… & x_{N}^{1}(n)\end{bmatrix}$$

计算参数

$$W = (B^{T} B)^{-1} B^TY,其中W=\begin{bmatrix} a\\ b\end{bmatrix}(N=1)；W=\begin{bmatrix} a\\ b_{2}\\ ……\\ b_{N}\end{bmatrix}(N\geq 2)$$

+ **STEP7:** 时间响应表达式：

(1)**当$N= 1$时：**
$$\hat{x}_{1}^{1}(t) = ({x}_{1}^{0}(1) - \frac{b}{a})e^{-a(t-1)} + \frac{b}{a}, t=1,2……,k,……$$


(2)**当$N\geq 2$时:**

$$\hat{x}_{1}^{1}(t) = [{x}_{1}^{0}(1) - \frac{1}{a} \sum_{i=2}^{N}b_{i}x^{1}_{i}(t)]e^{-a(k-1)} + \frac{1}{a} \sum_{i=2}^{N}b_{i}x^{1}_{i}(t), t=1,2……,k,……$$


目标序列的预测模型为：

$$ \hat{x}_{1}^{0}(1)={x}_{1}^{0}(1)$$$$ \hat{x}_{1}^{0}(t) = \hat{x}_{1}^{1}(t) - \hat{x}_{1}^{1}(t-1), t=2, 3, ……,n,……$$



### 3、模型效果检验

#### 3.1 残差

$$\varepsilon_{j} = {x}_{1}^{0}(j) - \hat{x}_{1}^{0}(j), j=2,3,……,n$$


#### 3.2 相对误差

$$\bigtriangleup_{j} = \frac{{x}_{1}^{0}(j) - \hat{x}_{1}^{0}(j)} {{x}_{1}^{0}(j)}, j=2,3,……,n$$

#### 3.3 平均相对误差

$$\epsilon_{j} = \frac{\sum_{j=2}^{n}|\bigtriangleup_{j}|}{n-1}$$

#### 3.4 方差比

$$c=\frac{残差序列方差}{原始数据序列方差}=\frac{S_{\varepsilon}}{S_{x_{1}^{0}(k)}}=\frac{\sqrt {\frac{1}{n}\sum_{j=1}^{n}(\varepsilon_{j} -\bar{\varepsilon})^{2}}}{\sqrt {\frac{1}{n}\sum_{j=1}^{n}(x_{1}^{0}(j) -\bar{x})^{2}}}$$

其中$$\bar{\varepsilon}=\frac{1}{n}\sum_{j=1}^{n}\varepsilon_{j},\bar{x}=\frac{1}{n}\sum_{j=1}^{n}x_{1}^{0}(j)$$

#### 3.5 小概率误差

$$p=P\{|\varepsilon_{j} - \bar{\varepsilon}|<0.6745S_{\varepsilon}\}$$


#### 模型效果评判


| 模型效果评判 |  $$p$$ | $$c$$ |
| :----: | :----: | :----: |
| 好   | $$0.95 \leq p$$ | $$c \leq 0.35$$ |
| 合格  | $$0.8 \leq p < 0.95$$ | $$0.35 <c \leq 0.5$$ |
| 勉强  | $$0.7 \leq p < 0.8$$ | $$0.5 < c \leq 0.65$$ |
| 不合格  | $$p<0.7$$ | $$ c > 0.65$$ |



### 4、Python3实例


In [ ]:
import pandas as pd
import numpy as np
from math import exp
from docx import Document
from lxml import etree
import latex2mathml.converter


def latex_to_word(latex_input):
    mathml = latex2mathml.converter.convert(latex_input)
    tree = etree.fromstring(mathml)
    xslt = etree.parse('MML2OMML.XSL')
    transform = etree.XSLT(xslt)
    new_dom = transform(tree)
    return new_dom.getroot()

In [2]:
class GM:
    
    def __init__(self, data, targetname, N=1):
        
        self.data = data  # 默认目标数据是第一列
        self.targetname= targetname
        self.N = N  # 建立GM(1,N)模型
        
    
    # 级比检验，不满足在计算平移参数
    def jibicheck(self):
        targetdata = self.data[self.targetname].values
        # jibi数据
        jibidata = targetdata[:-1] / targetdata[1:]
        
        # n
        length_n = len(targetdata)
        min_tap = e ** (-2/(n+1))
        max_tap = e ** (2/(n+1))
        
        checksign = 1
        multi = 0
        while checksign:
            # 计算平移参数
            new_c = round(multi * start_number)
            new_last_data = targetdata + new_c
            checksign = 0
            for kk in new_last_data:
                if kk <min_tap or kk > max_tap:
                    checksign = 1
            if not checksign:
                self.transc = new_c
                break
            multi += 0.01
        return print('级比检验')
    
    # 计算一次累加数据序列
    def computer_ago(self):
        
            
            
        



In [3]:
dd

{'中国':        年                           GDP
 0   2019  14.34万亿 (14,342,903,006,431)
 1   2018  13.89万亿 (13,894,817,549,374)
 2   2017  12.31万亿 (12,310,409,370,892)
 3   2016  11.23万亿 (11,233,276,536,737)
 4   2015  11.06万亿 (11,061,553,079,876)
 5   2014  10.48万亿 (10,475,682,920,594)
 6   2013    9.57万亿 (9,570,406,235,659)
 7   2012    8.53万亿 (8,532,229,986,993)
 8   2011    7.55万亿 (7,551,500,124,203)
 9   2010    6.09万亿 (6,087,163,874,512)
 10  2009     5.1万亿 (5,101,703,073,086)
 11  2008    4.59万亿 (4,594,307,032,660)
 12  2007    3.55万亿 (3,550,342,737,010)
 13  2006    2.75万亿 (2,752,131,773,355)
 14  2005    2.29万亿 (2,285,965,892,360)
 15  2004    1.96万亿 (1,955,347,004,963)
 16  2003    1.66万亿 (1,660,287,965,662)
 17  2002    1.47万亿 (1,470,550,015,081)
 18  2001    1.34万亿 (1,339,395,718,865),
 '美国':        年                           GDP
 0   2019  21.43万亿 (21,433,226,000,000)
 1   2018  20.58万亿 (20,580,159,776,000)
 2   2017  19.52万亿 (19,519,353,692,000)
 3   2016  18.71万亿 (18,714,

In [5]:
def getgdp(d):
    s = d.index('(')
    f = d.index(')')
    return d[s+1:f]

for k in dd:
    nh = dd[k]
    nh['GDP'] = nh['GDP'].apply(getgdp)
    nh = nh.sorted(byvalues='年')
    dd[k] = nh
    
print(dd)

ValueError: substring not found